## Imports

In [1]:
# from src.config.config import Config
# from src.db_writer.db import DB
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (12.8, 7.2)

## Preparation

In [3]:
df_fin = pd.read_csv('C:/Users/krotenko.n/Documents/GitHub/New folder/data/all_features.csv', index_col=[0])
df_fin.head()

,open_perp,close_perp,low_perp,high_perp,volume_perp,trades_perp,open_cq,close_cq,low_cq,high_cq,volume_cq,trades_cq,bidPrice_perp,bidQty_perp,askPrice_perp,askQty_perp,funding_rate,bidPrice_cq,bidQty_cq,askPrice_cq,askQty_cq,time_left,is_extrema,ismaxima,isminima,extr7,id_contract,day_of_week,second_of_day,open_close_diff_perp,high_low_diff_perp,avg_trade_volume_perp,bid_ask_spread_abs_perp,bid_ask_spread_perp,bid_ask_qty_spread_perp,bid_ask_qty_spread_abs_perp,bid_ask_qty_sum_perp,open_close_diff_cq,high_low_diff_cq,avg_trade_volume_cq,bid_ask_spread_abs_cq,bid_ask_spread_cq,bid_ask_qty_spread_cq,bid_ask_qty_spread_abs_cq,perp_cq_open_spread_abs,perp_cq_open_spread,short-term expectations_v1,short-term expectations_v2,short-term expectations_v3,short-term expectations_v4,short-term expectations_v5,short-term expectations_v6,short-term expectations_v7,short-term expectations_v8,short-term expectations_v9,symbol_BTCUSDT_221230,symbol_BTCUSDT_230331,symbol_BTCUSDT_230630,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,"('open_perp', 'min')","('open_perp', 'max')","('open_perp', 'minmax_diff')","('open_perp', 'minmax_rel_diff')","('open_cq', 'min')","('open_cq', 'max')","('open_cq', 'minmax_diff')","('open_cq', 'minmax_rel_diff')","('volume_perp', 'sum')","('volume_perp', 'mean')","('trades_perp', 'sum')","('trades_perp', 'mean')","('volume_cq', 'sum')","('volume_cq', 'mean')","('trades_cq', 'sum')","('trades_cq', 'mean')","('bidPrice_perp', 'min')","('bidPrice_perp', 'max')","('bidPrice_perp', 'minmax_diff')","('bidPrice_perp', 'minmax_rel_diff')","('askPrice_perp', 'min')","('askPrice_perp', 'max')","('askPrice_perp', 'minmax_diff')","('askPrice_perp', 'minmax_rel_diff')","('bidQty_perp', 'sum')","('bidQty_perp', 'mean')","('askQty_perp', 'sum')","('askQty_perp', 'mean')","('bidPrice_cq', 'min')","('bidPrice_cq', 'max')","('bidPrice_cq', 'minmax_diff')","('bidPrice_cq', 'minmax_rel_diff')","('askPrice_cq', 'min')","('askPrice_cq', 'max')","('askPrice_cq', 'minmax_diff')","('askPrice_cq', 'minmax_rel_diff')","('bidQty_cq', 'sum')","('bidQty_cq', 'mean')","('askQty_cq', 'sum')","('askQty_cq', 'mean')","('time_left', '<lambda>')"
0,21430.3,21445.0,21414.1,21445.0,619.156,5519,21459.9,21478.3,21440.6,21478.4,1.392,46,21432.6,0.820,21434.3,0.004,0.000012,21456.4,0.169,21462.8,0.071,3276000,True,False,True,False,0.0,1,36000,-14.7,30.9,0.112186,1.7,0.000079,-204.000000,-0.816,0.824,-18.4,37.8,0.030261,6.4,0.000298,-1.380282,-0.098,-29.6,-0.001381,-0.000009,-0.000007,-0.000003,0.000261,0.000216,0.000079,0.000002,0.000001,4.922718e-07,False,False,False,True,False,False,False,False,False,21363.3,21445.0,81.7,0.00381,21394.8,21479.4,84.6,0.003939,5375.086,767.869429,36447,5206.714286,17.641,2.520143,323,46.142857,21365.2,21444.9,79.7,0.003717,21365.3,21445.0,79.7,0.003716,101.623,14.517571,23.082,3.297429,21386.4,21476.7,90.3,0.004205,21392.6,21478.3,85.7,0.00399,0.703,0.100429,0.543,0.077571,420
1,21445.0,21445.0,21430.0,21446.8,311.295,3116,21476.7,21470.2,21460.0,21476.7,1.419,41,21444.9,25.023,21445.0,1.005,0.000011,21476.7,0.100,21478.3,0.054,3275940,True,True,False,True,0.0,1,36060,0.0,16.8,0.099902,0.1,0.000005,-23.898507,-24.018,26.028,6.5,16.7,0.034610,1.6,0.000074,-0.851852,-0.046,-31.7,-0.001478,-0.000009,-0.000008,-0.000003,0.000299,0.000248,0.000091,0.000002,0.000001,5.094377e-07,False,False,False,True,False,False,False,False,False,21363.3,21445.0,81.7,0.00381,21394.8,21479.4,84.6,0.003939,5375.086,767.869429,36447,5206.714286,17.641,2.520143,323,46.142857,21365.2,21444.9,79.7,0.003717,21365.3,21445.0,79.7,0.003716,101.623,14.517571,23.082,3.297429,21386.4,21476.7,90.3,0.004205,21392.6,21478.3,85.7,0.00399,0.703,0.100429,0.543,0.077571,420
2,21445.0,21431.2,21430.0,21448.6,250.049,2848,21479.4,21463.8,21460.4,21479.5,0.400,24,21444.9,17.361,21445.0,1.806,0.000010,21471.6,0.019,21476.5,0.121,3275880,False,False,False,False,0.0,1,36120,13.8,18.6,0.087798,0.1,0.000005,-8.612957,-15.555,19.167,15.6,19.1,0

Lets imagine our range of opening contract lies in range: 
$$\bigg[\frac{high + max(open, close)}{2} : \frac{low + min(open, close)}{2}\bigg]$$

In [4]:
df_fin['low_border'] = (df_fin['low_perp'] + df_fin[['open_perp', 'close_perp']].min(axis=1)) / 2
df_fin['up_border'] = (df_fin['high_perp'] + df_fin[['open_perp', 'close_perp']].max(axis=1)) / 2
df_fin.loc[df_fin['ismaxima'] & df_fin['extr7'], 'target1'] = df_fin.loc[df_fin['ismaxima'] & df_fin['extr7'], 'up_border']
df_fin.loc[df_fin['isminima'] & df_fin['extr7'], 'target1'] = df_fin.loc[df_fin['isminima'] & df_fin['extr7'], 'low_border']


df_fin['min_over_upcoming_30min'] = df_fin['low_border'][::-1].rolling(window=30).min()[::-1]
df_fin['max_over_upcoming_30min'] = df_fin['up_border' ][::-1].rolling(window=30).max()[::-1]
df_fin.loc[df_fin['ismaxima'] & df_fin['extr7'], 'target2'] = df_fin.loc[df_fin['ismaxima'] & df_fin['extr7'], 'min_over_upcoming_30min']
df_fin.loc[df_fin['isminima'] & df_fin['extr7'], 'target2'] = df_fin.loc[df_fin['isminima'] & df_fin['extr7'], 'max_over_upcoming_30min']


df_fin.loc[df_fin['ismaxima'] & df_fin['extr7'], 'contract_type'] = 1
df_fin.loc[df_fin['isminima'] & df_fin['extr7'], 'contract_type'] = 0

df_fin[['target1', 'target2', 'contract_type']] = df_fin[['target1', 'target2', 'contract_type']].shift(-1).fillna(method='bfill')
df_fin.dropna(inplace=True)

In [5]:
# df_fin.loc[~df_fin['extr7'], ['ismaxima', 'isminima']] = False

In [6]:
def custom_loss(row, target_col, pred_col):
    if row['extr7']:
        if row['contract_type'] == 1:
            if row[pred_col] < row['open_perp']:
                return np.sqrt(np.abs(row[target_col] - row['open_perp']))
            elif (row[pred_col] >= row['open_perp']) & (row[pred_col] <= row['high_perp']):
                return np.sqrt(np.abs(row[target_col] - row[pred_col]))
            else:
                return 10 * np.sqrt(np.abs(row[pred_col] - row[target_col]))
            
        elif row['contract_type'] == 0:
            if row[pred_col] > row['open_perp']:
                return np.sqrt(np.abs(row['open_perp'] - row[target_col]))
            elif (row[pred_col] <= row['open_perp']) & (row[pred_col] >= row['low_perp']):
                return np.sqrt(np.abs(row[target_col] - row[pred_col]))
            else:
                return 10 * np.sqrt(np.abs(row[target_col] - row[pred_col]))
    
    
    else:
        if row['contract_type'] == 1:
            if row[pred_col] <= row['high_perp']:
                return 10 * np.sqrt(np.abs(row[pred_col] - row[target_col]))
            else:
                return np.sqrt(np.abs(row[pred_col] - row[target_col]))
        
        elif row['contract_type'] == 0:
            if row[pred_col] >= row['low_perp']:
                return 10 * np.sqrt(np.abs(row[target_col] - row[pred_col]))
            else:
                return np.sqrt(np.abs(row[target_col] - row[pred_col]))

## Preprocessing

In [7]:
print(df_fin.shape)
df_fin.head()

(381403, 112)


,open_perp,close_perp,low_perp,high_perp,volume_perp,trades_perp,open_cq,close_cq,low_cq,high_cq,volume_cq,trades_cq,bidPrice_perp,bidQty_perp,askPrice_perp,askQty_perp,funding_rate,bidPrice_cq,bidQty_cq,askPrice_cq,askQty_cq,time_left,is_extrema,ismaxima,isminima,extr7,id_contract,day_of_week,second_of_day,open_close_diff_perp,high_low_diff_perp,avg_trade_volume_perp,bid_ask_spread_abs_perp,bid_ask_spread_perp,bid_ask_qty_spread_perp,bid_ask_qty_spread_abs_perp,bid_ask_qty_sum_perp,open_close_diff_cq,high_low_diff_cq,avg_trade_volume_cq,bid_ask_spread_abs_cq,bid_ask_spread_cq,bid_ask_qty_spread_cq,bid_ask_qty_spread_abs_cq,perp_cq_open_spread_abs,perp_cq_open_spread,short-term expectations_v1,short-term expectations_v2,short-term expectations_v3,short-term expectations_v4,short-term expectations_v5,short-term expectations_v6,short-term expectations_v7,short-term expectations_v8,short-term expectations_v9,symbol_BTCUSDT_221230,symbol_BTCUSDT_230331,symbol_BTCUSDT_230630,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,"('open_perp', 'min')","('open_perp', 'max')","('open_perp', 'minmax_diff')","('open_perp', 'minmax_rel_diff')","('open_cq', 'min')","('open_cq', 'max')","('open_cq', 'minmax_diff')","('open_cq', 'minmax_rel_diff')","('volume_perp', 'sum')","('volume_perp', 'mean')","('trades_perp', 'sum')","('trades_perp', 'mean')","('volume_cq', 'sum')","('volume_cq', 'mean')","('trades_cq', 'sum')","('trades_cq', 'mean')","('bidPrice_perp', 'min')","('bidPrice_perp', 'max')","('bidPrice_perp', 'minmax_diff')","('bidPrice_perp', 'minmax_rel_diff')","('askPrice_perp', 'min')","('askPrice_perp', 'max')","('askPrice_perp', 'minmax_diff')","('askPrice_perp', 'minmax_rel_diff')","('bidQty_perp', 'sum')","('bidQty_perp', 'mean')","('askQty_perp', 'sum')","('askQty_perp', 'mean')","('bidPrice_cq', 'min')","('bidPrice_cq', 'max')","('bidPrice_cq', 'minmax_diff')","('bidPrice_cq', 'minmax_rel_diff')","('askPrice_cq', 'min')","('askPrice_cq', 'max')","('askPrice_cq', 'minmax_diff')","('askPrice_cq', 'minmax_rel_diff')","('bidQty_cq', 'sum')","('bidQty_cq', 'mean')","('askQty_cq', 'sum')","('askQty_cq', 'mean')","('time_left', '<lambda>')",low_border,up_border,target1,min_over_upcoming_30min,max_over_upcoming_30min,target2,contract_type
0,21430.3,21445.0,21414.1,21445.0,619.156,5519,21459.9,21478.3,21440.6,21478.4,1.392,46,21432.6,0.820,21434.3,0.004,0.000012,21456.4,0.169,21462.8,0.071,3276000,True,False,True,False,0.0,1,36000,-14.7,30.9,0.112186,1.7,0.000079,-204.000000,-0.816,0.824,-18.4,37.8,0.030261,6.4,0.000298,-1.380282,-0.098,-29.6,-0.001381,-0.000009,-0.000007,-0.000003,0.000261,0.000216,0.000079,0.000002,0.000001,4.922718e-07,False,False,False,True,False,False,False,False,False,21363.3,21445.0,81.7,0.00381,21394.8,21479.4,84.6,0.003939,5375.086,767.869429,36447,5206.714286,17.641,2.520143,323,46.142857,21365.2,21444.9,79.7,0.003717,21365.3,21445.0,79.7,0.003716,101.623,14.517571,23.082,3.297429,21386.4,21476.7,90.3,0.004205,21392.6,21478.3,85.7,0.00399,0.703,0.100429,0.543,0.077571,420,21422.2,21445.00,21445.9,21319.95,21446.80,21319.95,1.0
1,21445.0,21445.0,21430.0,21446.8,311.295,3116,21476.7,21470.2,21460.0,21476.7,1.419,41,21444.9,25.023,21445.0,1.005,0.000011,21476.7,0.100,21478.3,0.054,3275940,True,True,False,True,0.0,1,36060,0.0,16.8,0.099902,0.1,0.000005,-23.898507,-24.018,26.028,6.5,16.7,0.034610,1.6,0.000074,-0.851852,-0.046,-31.7,-0.001478,-0.000009,-0.000008,-0.000003,0.000299,0.000248,0.000091,0.000002,0.000001,5.094377e-07,False,False,False,True,False,False,False,False,False,21363.3,21445.0,81.7,0.00381,21394.8,21479.4,84.6,0.003939,5375.086,767.869429,36447,5206.714286,17.641,2.520143,323,46.142857,21365.2,21444.9,79.7,0.003717,21365.3,21445.0,79.7,0.003716,101.623,14.517571,23.082,3.297429,21386.4,21476.7,90.3,0.004205,21392.6,21478.3,85.7,0.00399,0.703,0.100429,0.543,0.077571,420,21437.5,21445.90,21344.8,21319.95,21446.80,21397.70,0.0
2,21445.0,21431.2,21430.0,21448.6,250.049,2848,21479.4,

In [8]:
X = df_fin.drop(columns= ['target1', 'target2'])
y = df_fin[['target1', 'target2']]

In [9]:
from sklearn.preprocessing import StandardScaler

X_scaled = StandardScaler().fit_transform(X)
y_scaled = StandardScaler().fit_transform(y)

In [10]:
X_scaled.shape, y_scaled.shape

((381403, 110), (381403, 2))

## LSTM model

In [11]:
import numpy as np

from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn import model_selection

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.activations import relu
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from keras_tuner.tuners import GridSearch
from keras_tuner.engine.tuner import Tuner
from keras_tuner.oracles import BayesianOptimizationOracle

tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### prepare LSTM data

In [12]:
def create_lstm_data(X, y, look_back=60):
    arrays = [np.array(X[i : i+look_back]) for i in range(len(X) - look_back + 1)]
    lstm_data = np.stack(arrays)
    lstm_target = y[look_back - 1:]
    print('initial X shape: ', np.array(X).shape, 'y shape: ', y.shape, 'look_back: ', look_back)
    print('output X shape: ', lstm_data.shape, 'y shape:', lstm_target.shape)
    return lstm_data, lstm_target


In [13]:
# X_lstm, y_lstm = create_lstm_data(X_scaled, y)
# np.save('X_lstm_60.npy', X_lstm)
# np.save('y_lstm_60.npy', y_lstm)

X_lstm = np.load('X_lstm_60.npy')
# y_lstm = np.load('y_lstm_60.npy')
y_lstm = y_scaled[60 - 1:]

### Train-test-val split

In [14]:
# validation will be last ~21k, the rest will be split for Nested Cross-Validation
X_tt, X_val = X_lstm[:350000], X_lstm[350000:]
y_tt, y_val = y_lstm[:350000], y_lstm[350000:]

## model

In [15]:
# Define the LSTM and Dense layer architecture
def create_model(hp):
    model = Sequential()
    input_shape = (X_tt.shape[1], X_tt.shape[2])

    num_dense_layers = hp.Choice('num_dense_layers', values=[3, 4, 5])
    num_units = hp.Choice('num_units', values=[32,64,128])
    # use_regularization = hp.Choice('use_regularization', values=[True,False])
    use_regularization = False
    use_dropout = hp.Choice('use_dropout', values=[True,False])
    learning_rate = hp.Choice('learning_rate', values=[1e-6, 1e-7, 1e-8])

    model.add(LSTM(units=num_units, activation='relu', input_shape=(input_shape)))
    for _ in range(num_dense_layers):
        if use_regularization:
            model.add(Dense(units=num_units, activation='relu', kernel_regularizer=l1_l2()))
            if use_dropout:
                model.add(Dropout(0.2))
        else:
            model.add(Dense(units=num_units, activation='relu'))
            if use_dropout:
                model.add(Dropout(0.2))
    model.add(Dense(units=2, activation='linear'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')
    return model

In [16]:
callbacks = [
    # EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
]

In [17]:
class CVTuner(Tuner):
    def run_trial(self, trial, x, y, batch_size=4096, epochs=50):
        tscv = TimeSeriesSplit(5)
        val_losses = []
        for train_indices, test_indices in tscv.split(x):
            x_train, x_test = x[train_indices], x[test_indices]
            y_train, y_test = y[train_indices], y[test_indices]
            model = self.hypermodel.build(trial.hyperparameters)
            model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val), callbacks=callbacks)
            val_losses.append(model.evaluate(x_test, y_test))
        self.oracle.update_trial(trial.trial_id, {'val_loss': np.mean(val_losses)})
        self.save_model(trial.trial_id, model)

In [18]:
tuner = CVTuner(
    hypermodel=create_model,
    project_name = 'project2',
    oracle = BayesianOptimizationOracle(
        objective='val_loss',
        max_trials=40))

INFO:tensorflow:Reloading Tuner from .\project2\tuner0.json


In [19]:
tuner.search(X_tt, y_tt, batch_size=2048, epochs=10)


Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
4                 |3                 |num_dense_layers
128               |32                |num_units
0                 |1                 |use_dropout
1e-06             |1e-06             |learning_rate

Epoch 1/10
29/29 [==============================] - 3s 82ms/step - loss: 3.2195 - val_loss: 2.4533
Epoch 2/10
29/29 [==============================] - 2s 62ms/step - loss: 3.3332 - val_loss: 2.4565
Epoch 3/10
29/29 [==============================] - 2s 65ms/step - loss: 2.7319 - val_loss: 2.4591
Epoch 4/10
29/29 [==============================] - 2s 63ms/step - loss: 2.4194 - val_loss: 2.4614
Epoch 5/10
29/29 [==============================] - 2s 62ms/step - loss: 2.7082 - val_loss: 2.4636
Epoch 6/10
29/29 [==============================] - 2s 61ms/step - loss: 2.7626 - val_loss: 2.4664
Epoch 7/10
29/29 [==============================] - 2s 61ms/step - loss: 2.4113 - val_loss: 2.4692
Epoch 8/10
29/29 [===

Traceback (most recent call last):
  File "c:\Coding\miniconda3\envs\diploma_ml\lib\site-packages\keras_tuner\engine\base_tuner.py", line 270, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Coding\miniconda3\envs\diploma_ml\lib\site-packages\keras_tuner\engine\base_tuner.py", line 235, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\krotenko.n\AppData\Local\Temp\ipykernel_18320\1269940446.py", line 9, in run_trial
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val), callbacks=callbacks)
  File "c:\Coding\miniconda3\envs\diploma_ml\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "c:\Coding\miniconda3\envs\diploma_ml\lib\site-packages\tensorflow\python\framework\constant_op.py", line 102, in convert_to_eager_tensor
    return ops.EagerTensor(valu

RuntimeError: Number of consecutive failures excceeded the limit of 3.
Traceback (most recent call last):
  File "c:\Coding\miniconda3\envs\diploma_ml\lib\site-packages\keras_tuner\engine\base_tuner.py", line 270, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Coding\miniconda3\envs\diploma_ml\lib\site-packages\keras_tuner\engine\base_tuner.py", line 235, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\krotenko.n\AppData\Local\Temp\ipykernel_18320\1269940446.py", line 9, in run_trial
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val), callbacks=callbacks)
  File "c:\Coding\miniconda3\envs\diploma_ml\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "c:\Coding\miniconda3\envs\diploma_ml\lib\site-packages\tensorflow\python\framework\constant_op.py

In [ ]:
# # Define the Keras Tuner RandomSearch
# tuner = GridSearch(
#     create_model,
#     objective='val_loss',
#     max_trials=18,
#     executions_per_trial=2,
#     directory='my_dir',
#     project_name='my_project3'
# )

In [ ]:
# # Perform the hyperparameter search with time series cross-validation
# tscv = TimeSeriesSplit(n_splits = 4)
# for train_index, test_index in tscv.split(X_tt):
#     X_train, X_test = X_tt[train_index], X_tt[test_index]
#     y_train, y_test = y_tt[train_index], y_tt[test_index]
#     print(X_train.shape, X_test.shape)
#     tuner.search(X_train, 
#                  y_train,
#                  validation_data=(X_test, y_test),
#                  epochs=3, 
#                  batch_size=2048, 
#                  callbacks=callbacks,
#                  verbose=1)

Trial 5 Complete [00h 00m 20s]
val_loss: 618.8718242645264

Best val_loss So Far: 0.4346502274274826
Total elapsed time: 00h 01m 41s

Search: Running Trial #6

Value             |Best Value So Far |Hyperparameter
3                 |3                 |num_dense_layers
32                |32                |num_units
0                 |0                 |use_dropout
1e-06             |0.0001            |learning_rate

Epoch 1/3
35/35 [==============================] - 4s 100ms/step - loss: 3.4149 - val_loss: 108.6216
Epoch 2/3
 5/35 [===>..........................] - ETA: 1s - loss: 2.7429

KeyboardInterrupt: 

## 1D-CNN model 

## Archive

In [ ]:
# def calculate_custom_metric(df, target_col, pred_values):
#     df_copy = df.copy()
#     df_copy['pred_col'] = pred_values
#     df_copy['pred_col'] = df_copy['pred_col'].shift(1)
#     return df_copy.apply(lambda row: custom_metric(row, target_col, 'pred_col'), axis=1).mean()

In [ ]:
# @tf.function
# def calculate_custom_loss(true_values , pred_values):
#     df_copy = df_fin.copy()
#     df_copy['pred_col'] = pred_values
#     df_copy['pred_col'] = df_copy['pred_col'].shift(1)
#     return df_copy.apply(lambda row: custom_loss(row, 'target1', 'pred_col'), axis=1).mean()

In [ ]:
# model = KerasRegressor(
#     model=create_model, 
#     # validation_data=(X_val, y_val), 
#     epochs=20, 
#     batch_size=2048, 
#     callbacks=callbacks,
#     verbose=1
# )

In [ ]:
# # Define the hyperparameters to search over
# param_grid = {
#     'num_dense_layers': [2, 3, 4],
#     'num_units': [32, 64, 128],
#     'use_regularization': [True, False],
#     'use_dropout': [True, False],
#     'learning_rate': [0.1, 0.05, 0.01, 0.005, 0.001]
# }

In [ ]:
# grid_search = GridSearchCV(
#     estimator=model, 
#     param_grid=param_grid, 
#     cv=tscv, 
#     scoring='neg_mean_squared_error', 
#     verbose=1, 
#     n_jobs=-2)
# grid_search.fit(X_tt, y_tt)